In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from itertools import izip, count

In [65]:
"""
Read in train and test as Pandas DataFrames
"""
df_train = pd.read_csv("trainVirus.csv")
df_test = pd.read_csv("testVirus.csv")

In [66]:
df_train.head()

,filename,virus,CREATE_SUSPENDED,WININET.dll.InternetOpenUrl,WS_MAXIMIZEBOX,OverlappedWinsockIO_Recv,<dump_line,Einstellungen,&#xD9,NetShareEnumSticky,MSWSOCK,00 00 00 00 00 00 00 00,Travel Insurance.url,MachineGuid,WINHELP.INI,Shell_TrayWnd
0,00269ea50001a6c699d0222032d45b74b2e7e8be9.None...,8,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,00278ec420236020d6121dffe0cc20034422e7228.Lipl...,6,0,12,1,0,3434,295,0,12,0,27,0,1,0,3
2,002d5615d19c851934dc481c607b6a74a6e9e536e.VB.xml,12,0,0,1,0,0,0,0,0,0,0,0,0,2,0
3,006be5Dc265600c19728c9747fb4c7bc9e8d6f106.None...,8,0,0,1,0,0,116,0,2,0,0,0,0,0,1
4,0089453df77890cae95ce7d9130a4ef85eaea36e8.Swiz...,10,3,0,1,32,636,0,0,0,0,0,0,0,0,0


In [67]:
df_test.head()

,filename,CREATE_SUSPENDED,WININET.dll.InternetOpenUrl,WS_MAXIMIZEBOX,OverlappedWinsockIO_Recv,<dump_line,Einstellungen,&#xD9,NetShareEnumSticky,MSWSOCK,00 00 00 00 00 00 00 00,Travel Insurance.url,MachineGuid,WINHELP.INI,Shell_TrayWnd
0,0015c8c9ff02fea9d0f45692b9eebfb4abff4e42f,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,001f298a534ae4b0db7f2707169250aa215c3b5f2,3,4,0,0,31,4,0,2,0,0,0,0,0,0
2,001f5fdaaa8bbe20303527198d09a30bb7ca3eb50,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,002ca2c41b649f85c05ae30013436781a932fecc6,0,4,0,0,280,45,0,2,0,0,0,1,0,1
4,003e109543b4ea22d2bcc1ec309bf2fd34e9a1a1d,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [68]:
#store gap values
Y_train = df_train.virus.values
#store test file names
testNames = df_test.filename.values
#row where testing examples start
test_idx = df_train.shape[0]
#delete 'Id' column
#df_test = df_test.drop(['virus'], axis=1)
#delete 'gap' column
df_train = df_train.drop(['virus'], axis=1)

In [69]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
df_all = pd.concat((df_train, df_test), axis=0)
df_all.head()

,filename,CREATE_SUSPENDED,WININET.dll.InternetOpenUrl,WS_MAXIMIZEBOX,OverlappedWinsockIO_Recv,<dump_line,Einstellungen,&#xD9,NetShareEnumSticky,MSWSOCK,00 00 00 00 00 00 00 00,Travel Insurance.url,MachineGuid,WINHELP.INI,Shell_TrayWnd
0,00269ea50001a6c699d0222032d45b74b2e7e8be9.None...,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,00278ec420236020d6121dffe0cc20034422e7228.Lipl...,0,12,1,0,3434,295,0,12,0,27,0,1,0,3
2,002d5615d19c851934dc481c607b6a74a6e9e536e.VB.xml,0,0,1,0,0,0,0,0,0,0,0,0,2,0
3,006be5Dc265600c19728c9747fb4c7bc9e8d6f106.None...,0,0,1,0,0,116,0,2,0,0,0,0,0,1
4,0089453df77890cae95ce7d9130a4ef85eaea36e8.Swiz...,3,0,1,32,636,0,0,0,0,0,0,0,0,0


In [70]:
#Drop the 'smiles' column
df_all = df_all.drop(['filename'], axis=1)
vals = df_all.values
X_train = vals[:test_idx]
X_test = vals[test_idx:]
print "Train features:", X_train.shape
print "Train gap:", Y_train.shape
print "Test features:", X_test.shape

Train features: (3020, 14)
Train gap: (3020,)
Test features: (3363, 14)


In [71]:
RFC = RandomForestClassifier()
RFC.fit(X_train, Y_train)
RFC_pred = RFC.predict(X_test)

In [72]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        #for i,p in enumerate(predictions):
        #    f.write(str(i+1) + "," + str(p) + "\n")     
        for i,j in izip(testNames, predictions):
            f.write(str(i) + "," + str(j) + "\n")  

In [73]:
write_to_file("output.csv", RFC_pred)